In [1]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

In [2]:
import os

from ipywidgets import interact
from confapp import conf
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display
from natsort import natsorted

from mic_analysis.plot_interactive import display_partition_figures, get_possible_variables, get_possible_outliers, visualize_outlier, visualize_video

Loaded local_settings to conf


# Visualize pre-plotted figures

In [3]:
videos_table = pd.read_csv(conf.VIDEOS_INDEX_FILE_NAME)

partition_col = "line_experiment"
video_uid_col = "trial_uid"

possible_partition_uids = natsorted(videos_table[partition_col].unique())

save_path = os.path.join(conf.GENERATED_FIGURES_PATH, f"summary_{partition_col}")
if not os.path.isdir(save_path):
    os.makedirs(save_path)


@interact(
    partition_uid=possible_partition_uids,
    var_type=["indiv", "group", "indiv_nb"],
    visualize_outliers=False,
    visualize_videos=False,
)
def summary_partition(
    partition_uid="ap1g1_1_1", 
    var_type="indiv", 
):

    load_path = os.path.join(save_path, partition_uid)
    display_partition_figures(load_path, var_type)
    
    videos_summary_figs_path = os.path.join(load_path, "per_video_summaries")
    
    partition_videos = videos_table[(videos_table[partition_col] == partition_uid) & videos_table.valid_for_analysis]
    genotype_groups = partition_videos.genotype_group.unique()
    @interact(genotype_group=genotype_groups)
    def select_genotype(genotype_group):
        genotype_videos = partition_videos[partition_videos.genotype_group == genotype_group]
        partition_genotype_videos_uids = genotype_videos.trial_uid.unique()
        
        @interact(video_uid=partition_genotype_videos_uids)
        def show_video(video_uid):
            visualize_video(
                videos_summary_figs_path, videos_table, video_uid_col, video_uid, var_type
            )

            outliers_path = os.path.join(load_path, "all_outliers.csv")
            if os.path.isfile(outliers_path):
                outliers = pd.read_csv(outliers_path)
                outliers_per_variable_type = outliers[outliers.var_type == var_type]
                variables = get_possible_variables(outliers_per_variable_type)
                

                @interact(variable=variables)
                def show_variable(variable):
                    outliers_uids = get_possible_outliers(outliers_per_variable_type, variable, video_uid_col)

                    @interact(outlier=outliers_uids)
                    def plot_outlier_var(outlier):
                        visualize_outlier(
                            outliers_per_variable_type, outlier, videos_summary_figs_path, variable, video_uid_col
                        )


interactive(children=(Dropdown(description='partition_uid', options=('ap1g1_1_1', 'apbb1_1_1', 'asap1b_7_1', '…

interactive(children=(Dropdown(description='variable', options=("('normed_distance_to_origin', 'median')", "('…

In [30]:
videos_table[videos_table.trial_uid == 'ap1g1_1_1_1_1']

,trial_uid,trial,px_cm,accuracy,file_name,folder_name_track,genotype_group,group_size,old_name_experiment,social_data_old,...,valid_num_impossible_speed_jumps,valid_num_unsolvable_impossible_speed_jumps,valid_id_last_fish,valid_certainty_id_last_fish,tracking_state,id_last_fish_state,for_analysis_state,valid_tracking,valid_genotype_id,valid_for_analysis
1047,ap1g1_1_1_1_1,1,NaN,0.9969,20200709_ap1g1_1_1_1.csv,ap1g1_1_1_1,HET_HET,2,ap1g1_1,ap1g1_1,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True


In [39]:
videos_table[videos_table.trial_uid.str.contains('srrm3_17_1')]

,trial_uid,trial,px_cm,accuracy,file_name,folder_name_track,genotype_group,group_size,old_name_experiment,social_data_old,...,valid_num_impossible_speed_jumps,valid_num_unsolvable_impossible_speed_jumps,valid_id_last_fish,valid_certainty_id_last_fish,tracking_state,id_last_fish_state,for_analysis_state,valid_tracking,valid_genotype_id,valid_for_analysis
94,srrm3_17_1_1_1,1,NaN,0.9859,20200826_srrm3_17_1_1.csv,srrm3_17_1_1,WT_HET,2,Srrm3_17_1,srrm3_17_2,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
95,srrm3_17_1_1_2,2,NaN,0.9913,20200826_srrm3_17_1_1.csv,srrm3_17_1_1,WT_HET,2,Srrm3_17_1,srrm3_17_2,...,True,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
96,srrm3_17_1_1_3,3,NaN,0.9966,20200826_srrm3_17_1_1.csv,srrm3_17_1_1,WT_HET,2,Srrm3_17_1,srrm3_17_2,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
97,srrm3_17_1_1_4,4,NaN,0.9846,20200826_srrm3_17_1_1.csv,srrm3_17_1_1,WT_HET,2,Srrm3_17_1,srrm3_17_2,...,True,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
98,srrm3_17_1_1_5,5,NaN,0.9857,20200826_srrm3_17_1_1.csv,srrm3_17_1_1,HET_HET,2,Srrm3_17_1,srrm3_17_2,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,srrm3_17_1_2_6,6,NaN,0.9918,20200317_srrm3_17_1_2.csv,srrm3_17_1_2,DEL_DEL,2,Srrm3_17,srrm3_17_OK,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
1363,srrm3_17_1_2_7,7,NaN,0.9958,20200317_srrm3_17_1_2.csv,srrm3_17_1_2,DEL_DEL,2,Srrm3_17,srrm3_17_OK,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
1364,srrm3_17_1_2_8,8,NaN,0.9989,20200317_srrm3_17_1_2.csv,srrm3_17_1_2,DEL_DEL,2,Srrm3_17,srrm3_17_OK,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True
1365,srrm3_17_1_2_9,9,NaN,0.9953,20200317_srrm3_17_1_2.csv,srrm3_17_1_2,DEL_DEL,2,Srrm3_17,srrm3_17_OK,...,False,True,True,True,1 1 1 1 1,1 1,1 1 1 1 1-1 1,True,True,True


In [47]:
videos_table.line_experiment.value_counts()

srrm3_17_1             133
gli3_7_1               106
srrm4_12_1             100
shank3a_2_1             97
vti1a_1_1               76
vav2_1_1                76
gli2b_1_1               72
vdac3_5_1               72
evi5b_6_1               68
reln_2_1                48
itsn1_1_1               48
shank3b_1_1             48
cdon_1_1                46
srrm4-srrm3_17-12_1     44
mef2ca_2_1              42
src_1_1                 40
vav2_2_1                40
itsn1_3_1               38
kif1b_1_1               34
ap1g1_1_1               32
asap1b_7_1              28
srrm3_5_5               28
evi5b_3_1               25
apbb1_1_1               25
madd_1_1                24
srrm3_17_2              10
srrm3_17_3               9
srrm3_17_4               6
Name: line_experiment, dtype: int64